# Import Dependenciesg

In [2]:
import os

import tensorflow as tf
import pandas as pd
import keras

2022-12-13 21:47:58.250384: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-13 21:47:59.116967: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-12-13 21:47:59.117007: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2022-12-13 21:47:59.117011: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [ ]:
keras.__version__

In [ ]:
tf.__version__

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Read in the Data 

In [ ]:
rawData = pd.read_excel("Full-Data-Set-UnderOver-2020-21.xlsx")
rawData = rawData.sample(frac=1)

In [ ]:
rawData.describe()

## Clean the Data

In [ ]:
filteredData = rawData.drop(["Unnamed: 0"], axis = 1)
filteredData = filteredData.drop(["Score"], axis = 1)
filteredData = filteredData.drop(["GP"], axis = 1)
filteredData = filteredData.drop(["GP.1"], axis = 1)
filteredData = filteredData.drop(["Home-Team-Win"], axis = 1)
filteredData = filteredData.drop(["TEAM_NAME"], axis = 1)
filteredData = filteredData.drop(["Date"], axis = 1)
filteredData = filteredData.drop(["MIN", "W", "W.1", "L", "L.1", "PLUS_MINUS", "PLUS_MINUS.1", "PLUS_MINUS_RANK", "PLUS_MINUS_RANK.1", "W_RANK", "W_RANK.1", "L_RANK", "L_RANK.1"], axis = 1)
filteredData = filteredData.drop(["MIN.1"], axis = 1)
filteredData = filteredData.drop(["MIN_RANK.1"], axis = 1)
filteredData = filteredData.drop(["MIN_RANK"], axis = 1)
filteredData = filteredData.drop(["TEAM_NAME.1"], axis = 1)
filteredData = filteredData.drop(["Date.1"], axis = 1)
filteredData = filteredData.drop(["OU"], axis = 1)
filteredData = filteredData.drop(["OU-Cover"], axis = 1)
filteredData = filteredData.drop(["GP_RANK"], axis = 1)
filteredData = filteredData.drop(["GP_RANK.1"], axis = 1)

# Train the Model

#### Get the outputs

In [ ]:
output = rawData['Home-Team-Win']

In [ ]:
#get 80% for training 20% for testing
print(len(filteredData))
test_len = int(len(filteredData) * 0.8)
x_train = filteredData.take(range(test_len))
x_test = filteredData.take(range(test_len, len(filteredData)))
y_train = output.take(range(test_len))
y_test = output.take(range(test_len, len(filteredData)))

In [ ]:
test_len = int(len(filteredData) * 0.9)
x_train = filteredData.take(range(test_len))
x_test = filteredData.take(range(test_len, len(filteredData)))
y_train = output.take(range(test_len))
y_test = output.take(range(test_len, len(filteredData)))

In [ ]:
import datetime
%load_ext tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(170, activation=tf.nn.relu6))
model.add(tf.keras.layers.Dropout(rate=.25))
model.add(tf.keras.layers.Dense(150, activation=tf.nn.relu6))
model.add(tf.keras.layers.Dense(75, activation=tf.nn.relu6))
model.add(tf.keras.layers.Dropout(rate=.15))
model.add(tf.keras.layers.Dense(30, activation=tf.nn.relu6))
model.add(tf.keras.layers.Dense(15, activation=tf.nn.relu6))
model.add(tf.keras.layers.Dense(2, activation=tf.nn.softmax, name="output_layer"))


In [ ]:
model.build( x_train.shape )
model.summary()

In [ ]:
from keras.optimizers import Adam
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
model.compile(Adam(learning_rate=0.0005), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, epochs=60, validation_split=0.30, batch_size=3_000, verbose=2, shuffle=True, workers=15, use_multiprocessing=True, callbacks=[tensorboard_callback])

In [ ]:
todayPredictions = model.predict([x_test])

In [3]:
model = tf.keras.models.load_model("./trained_models/v2")

2022-12-13 21:48:34.038266: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-13 21:48:34.055172: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-13 21:48:34.055316: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-13 21:48:34.055767: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

In [14]:
import os
import numpy as np
model_dir = "../API/src/data"
for file in os.listdir(model_dir):
    if not file.endswith(".csv") or not file.__contains__("2022"):
        continue

    teamStats = pd.read_csv(f"{model_dir}/{file}")
    filteredTeamStats = teamStats.drop(["TEAM_NAME", "TEAM_ID","GP","GP_RANK","CFID", "MIN", "CFPARAMS", "MIN_RANK", "MIN_RANK.1"], axis = 1)
    filteredTeamStats = filteredTeamStats.drop(["W","L","PLUS_MINUS","W_RANK", "L_RANK", "PLUS_MINUS_RANK",], axis = 1)
    filteredTeamStats = filteredTeamStats.drop(["TEAM_NAME.1", "TEAM_ID.1","GP.1","GP_RANK.1","CFID.1", "MIN.1", "CFPARAMS.1",], axis = 1)
    filteredTeamStats = filteredTeamStats.drop(["W.1","L.1","PLUS_MINUS.1","W_RANK.1", "L_RANK.1", "PLUS_MINUS_RANK.1",], axis = 1)
    todayPredictions = model.predict(filteredTeamStats)

    print(f"{file}")
    for i in range (0,len(todayPredictions)):
        away_team_perc = todayPredictions[i][0]
        home_team_perc = todayPredictions[i][1]
        print(f"{teamStats['TEAM_NAME'][i]} {home_team_perc} vs {teamStats['TEAM_NAME.1'][i]} {away_team_perc}")
        if np.argmax(todayPredictions[i]) == 1:
            print(f"{teamStats['TEAM_NAME'][i]} win")
        else:
            print(f"{teamStats['TEAM_NAME.1'][i]} win ")
        print()
        print()


1/1 [==============================] - 0s 17ms/step
2022-11-23.csv
Cleveland Cavaliers 0.842477023601532 vs Portland Trail Blazers 0.15752290189266205
Cleveland Cavaliers win


Atlanta Hawks 0.6489821076393127 vs Sacramento Kings 0.35101792216300964
Atlanta Hawks win


Boston Celtics 0.863115131855011 vs Dallas Mavericks 0.1368848830461502
Boston Celtics win


Oklahoma City Thunder 0.22785867750644684 vs Denver Nuggets 0.7721412777900696
Denver Nuggets win 


Charlotte Hornets 0.36465877294540405 vs Philadelphia 76ers 0.6353411674499512
Philadelphia 76ers win 


Utah Jazz 0.903127908706665 vs Detroit Pistons 0.09687205404043198
Utah Jazz win


Miami Heat 0.3807199001312256 vs Washington Wizards 0.6192800402641296
Washington Wizards win 


Indiana Pacers 0.7325899600982666 vs Minnesota Timberwolves 0.267410010099411
Indiana Pacers win


San Antonio Spurs 0.24451738595962524 vs New Orleans Pelicans 0.75548255443573
New Orleans Pelicans win 


Toronto Raptors 0.6884763240814209 vs Brookly

# Save the Model

In [ ]:
import datetime
model.save(f"model_{datetime.datetime.utcnow()}")